In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
# Set Up Variables
new_df = application_df.drop(['EIN','NAME'], axis = 1)
unique = new_df.nunique()
selected_cols = [col for col in list(dict(unique).keys()) if dict(unique)[col] > 10][:2]
application_counts = new_df[selected_cols[0]].value_counts()
classification_counts = new_df[selected_cols[1]].value_counts()

In [ ]:
# Bin Application Column and Classifications Column
# -------------------------------------------------
# Application Set Up
# ------------------
cut_off = 156
application_types_to_replace = [type for type in list(dict(application_counts).keys()) if dict(application_counts)[type] <= cut_off ]

for app in application_types_to_replace:
    new_df['APPLICATION_TYPE'] = new_df['APPLICATION_TYPE'].replace(app,"Other")


# Classifications Set Up
# ----------------------
classifications_to_replace = []
min_cut_off = 1
cut_off = 1883

for type in [x for x in list(dict(classification_counts).keys())]:
    if dict(classification_counts)[type] < cut_off or dict(classification_counts)[type] == min_cut_off:
        classifications_to_replace.append(type)

for cls in classifications_to_replace:
    new_df['CLASSIFICATION'] = new_df['CLASSIFICATION'].replace(cls,"Other")


In [ ]:
# Set up final Dataframe that will used for model training and evaluation
# -----------------------------------------------------------------------
dum_df = pd.get_dummies(new_df)
target = dum_df['IS_SUCCESSFUL']
features = dum_df.drop(['IS_SUCCESSFUL'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(features,target)

# Scale
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Convert to Numpy Arrays to avoid data type issue with TensorFlow
scale_train_x = np.asarray(X_train_scaled)
train_y = np.asarray(y_train)
scale_test_x = np.asarray(X_test_scaled)
test_y = np.asarray(y_test)

In [ ]:
# Define and fir the deep neural net model 
# ---------------------------------
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

# fit the model 
fit_model = nn_model.fit(scale_train_x, train_y, epochs=100)

In [ ]:
# Evaluate the Model and Save to h5 file
# --------------------------------------
# Evaluate
model_loss, model_accuracy = nn_model.evaluate(scale_test_x ,test_y,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save and export to h5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")

